In [27]:
import pandas as pd
import google.generativeai as genai
import os
import sqlite3
import fitz # PyMuPDF
import json
import re

In [28]:
try:
    genai.configure(api_key=os.environ.get("GEMINI_API_KEY") or "YOUR_API_KEY")

    # Attempt to list models as a basic connection test
    # for model in genai.list_models():
    #     if 'generateContent' in model.supported_generation_methods:
    #         print(f"Successfully connected to Gemini API. Found model: {model.name}")

    model = genai.GenerativeModel('gemini-2.5-flash') 
    response = model.generate_content("What is the capital of France?")
    print("\n--- Direct Content Generation Test ---")
    print(f"Response from Gemini: {response.text}")
    print("Connection test successful!")

except Exception as e:
    print(f"Connection test failed: {e}")
    print("Please check your API key, network connection, and ensure the Gemini API is enabled for your project.")


--- Direct Content Generation Test ---
Response from Gemini: The capital of France is Paris.
Connection test successful!


In [ ]:


# Optional: For SQL magic commands in Jupyter
%load_ext sql

In [29]:
db_file = 'uniformat_data.db'
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

print(f"Connected to SQLite database: {db_file}")

# Create the UniformatCode table
cursor.execute('''
CREATE TABLE IF NOT EXISTS UniformatCode (
    UniformatCodeID INTEGER PRIMARY KEY AUTOINCREMENT,
    UniformatCode VARCHAR(20) NOT NULL UNIQUE,
    Description VARCHAR(500) NOT NULL,
    LongDescription TEXT,
    ParentCodeID INTEGER,
    Level INTEGER NOT NULL,
    IsActive BOOLEAN DEFAULT TRUE,
    StandardVersion VARCHAR(50),
    CreatedDate DATETIME DEFAULT CURRENT_TIMESTAMP,
    LastUpdatedDate DATETIME DEFAULT CURRENT_TIMESTAMP
);
''')
conn.commit()
print("UniformatCode table created or already exists.")

# Configure ipython-sql to use this connection
%sql sqlite:///uniformat_data.db

Connected to SQLite database: uniformat_data.db
UniformatCode table created or already exists.


In [ ]:
# # Load your CSV file
# try:
#     df_csv = pd.read_excel('Uniformat-II-Levels-1-to-5.xlsx', keep_default_na=False) # keep_default_na=False to treat empty strings as empty, not NaN
#     print("CSV loaded successfully.")
#     print(df_csv.head(15)) # Show more rows to see the fill logic in action
# except FileNotFoundError:
#     print("Error: 'Uniformat-II-Levels-1-to-5.xlsx' not found. Please provide your actual CSV file.")
#     # Exit or handle error appropriately


# df_csv.columns = df_csv.columns.str.strip()

# # --- Step 1: Forward-fill the parent columns ---
# # This propagates the higher-level codes down to their children
# df_csv['Level 1'] = df_csv['Level 1'].replace('', pd.NA).ffill()
# df_csv['Level 2'] = df_csv['Level 2'].replace('', pd.NA).ffill()
# df_csv['Level 3'] = df_csv['Level 3'].replace('', pd.NA).ffill()
# df_csv['Uniformat Level 4'] = df_csv['Uniformat Level 4'].replace('', pd.NA).ffill()
# df_csv['Uniformat Level 5'] = df_csv['Uniformat Level 5'].replace('', pd.NA) # Level 5 should not ffill, it's the deepest

# print("\nDataFrame after forward-filling:")
# print(df_csv.head(15))

# # --- Step 2: Determine the actual Uniformat code and its level for each row ---
# # This is tricky because the 'Description' column aligns with the deepest *filled* code on that row.
# processed_uniformat_data = []
# for index, row in df_csv.iterrows():
#     code = None
#     level = None
#     parent_code_str = None
#     description = row['Description']

#     # We need to find the deepest non-empty code on this specific row
#     if pd.notna(row['Uniformat Level 5']):
#         code = row['Uniformat Level 5']
#         level = 5
#         # Parent of L5 is L4
#         parent_code_str = row['Uniformat Level 4'] # This is the immediate parent from the filled column
#     elif pd.notna(row['Uniformat Level 4']):
#         code = row['Uniformat Level 4']
#         level = 4
#         # Parent of L4 is L3
#         parent_code_str = row['Level 3']
#     elif pd.notna(row['Level 3']):
#         code = row['Level 3']
#         level = 3
#         # Parent of L3 is L2
#         parent_code_str = row['Level 2']
#     elif pd.notna(row['Level 2']):
#         code = row['Level 2']
#         level = 2
#         # Parent of L2 is L1
#         parent_code_str = row['Level 1']
#     elif pd.notna(row['Level 1']):
#         code = row['Level 1']
#         level = 1
#         parent_code_str = None # Top level

#     if code: # Only add if a valid code was found for the row
#         processed_uniformat_data.append({
#             'code': code.strip(), # Clean whitespace
#             'description': description.strip(),
#             'long_description': description.strip(), # Initial placeholder
#             'level': level,
#             'parent_code_str': parent_code_str.strip() if parent_code_str else None,
#             'standard_version': 'Custom' if level == 5 else 'ASTM E1557-09'
#         })

# # Convert to DataFrame for easier inspection/deduplication
# df_processed_final = pd.DataFrame(processed_uniformat_data).drop_duplicates(subset=['code']).reset_index(drop=True)
# print("\nFinal processed Uniformat data ready for DB insertion (deduplicated):")
# print(df_processed_final.head(15))

# # --- Step 3: Insert into SQLite ---
# # This part now uses the `df_processed_final` DataFrame

# cursor = conn.cursor() # Ensure cursor is active

# # Get existing codes in DB to prevent re-inserting
# cursor.execute("SELECT UniformatCode, UniformatCodeID FROM UniformatCode")
# db_codes_map = {row[0]: row[1] for row in cursor.fetchall()}

# # Create a list for records to insert, sorted by level to ensure parents exist
# records_to_insert = df_processed_final.to_dict(orient='records')
# records_to_insert.sort(key=lambda x: x['level'])

# # First pass: Insert all codes, mapping code string to DB ID
# for record in records_to_insert:
#     if record['code'] not in db_codes_map:
#         try:
#             cursor.execute('''
#                 INSERT INTO UniformatCode (UniformatCode, Description, LongDescription, Level, StandardVersion)
#                 VALUES (?, ?, ?, ?, ?)
#             ''', (record['code'], record['description'], record['long_description'], record['level'], record['standard_version']))
#             conn.commit()
#             db_codes_map[record['code']] = cursor.lastrowid # Update map with new ID
#         except sqlite3.IntegrityError as e:
#             print(f"Skipping duplicate or error inserting {record['code']}: {e}")
#             conn.rollback()
#             # If it's a unique constraint violation, it means it's already there,
#             # so get its ID for later parent mapping.
#             cursor.execute("SELECT UniformatCodeID FROM UniformatCode WHERE UniformatCode = ?", (record['code'],))
#             result = cursor.fetchone()
#             if result:
#                 db_codes_map[record['code']] = result[0]
#             else:
#                 print(f"Failed to retrieve ID for existing code {record['code']}")


# # Second pass: Update ParentCodeID using the collected IDs
# for record in records_to_insert:
#     current_uniformat_id = db_codes_map.get(record['code'])
#     parent_uniformat_id = db_codes_map.get(record['parent_code_str']) # Get ID of parent code string

#     if current_uniformat_id and record['parent_code_str'] and parent_uniformat_id:
#         # Check if ParentCodeID is already set to avoid unnecessary updates
#         cursor.execute("SELECT ParentCodeID FROM UniformatCode WHERE UniformatCodeID = ?", (current_uniformat_id,))
#         current_parent_id_in_db = cursor.fetchone()[0]

#         if current_parent_id_in_db != parent_uniformat_id:
#             try:
#                 cursor.execute('''
#                     UPDATE UniformatCode
#                     SET ParentCodeID = ?
#                     WHERE UniformatCodeID = ?
#                 ''', (parent_uniformat_id, current_uniformat_id))
#                 conn.commit()
#                 # print(f"Updated ParentCodeID for {record['code']} to {record['parent_code_str']}")
#             except Exception as e:
#                 print(f"Error updating ParentCodeID for {record['code']}: {e}")

# print("\nUniformat codes (Levels 1-5) from CSV inserted and parent IDs resolved in database.")
# print("\nVerifying hierarchy in DB:")


CSV loaded successfully.
                 Level 1    Unnamed: 1         Level 2   Unnamed: 3  \
0   Major Group Elements                Group Elements                
1                      A  SUBSTRUCTURE                                
2                                                  A10  Foundations   
3                                                                     
4                                                                     
5                                                                     
6                                                                     
7                                                                     
8                                                                     
9                                                                     
10                                                                    
11                                                                    
12                                                  

In [30]:
# Instead of:
# %sql SELECT UniformatCode, Description, Level, ParentCodeID, (SELECT UniformatCode FROM UniformatCode AS P WHERE P.UniformatCodeID = UC.ParentCodeID) AS ParentCodeStr FROM UniformatCode AS UC ORDER BY UniformatCode;

# Use this Python code with pandas:
try:
    # Fetch data directly into a pandas DataFrame
    df_uniformat_hierarchy = pd.read_sql_query('''
        SELECT
            UC.UniformatCode,
            UC.Description,
            UC.Level,
            UC.ParentCodeID,
            P.UniformatCode AS ParentCodeStr
        FROM
            UniformatCode AS UC
        LEFT JOIN
            UniformatCode AS P ON UC.ParentCodeID = P.UniformatCodeID
        ORDER BY
            UC.UniformatCode;
    ''', conn) # 'conn' is your sqlite3 connection object

    print("\nVerifying hierarchy in DB (using pandas):")
    print(df_uniformat_hierarchy.head(20)) # Display the first 20 rows

except Exception as e:
    print(f"Error fetching data to verify hierarchy: {e}")


Verifying hierarchy in DB (using pandas):
Empty DataFrame
Columns: [UniformatCode, Description, Level, ParentCodeID, ParentCodeStr]
Index: []


In [31]:
import pandas as pd
import sqlite3
import re
# ... other imports ...

# Assuming your CSV file is named file
try:
    df_csv = pd.read_excel('Uniformat-II-Levels-1-to-5.xlsx', keep_default_na=False)
    print("CSV loaded successfully.")

    # --- IMMEDIATE COLUMN CLEANING ---
    df_csv.columns = df_csv.columns.str.strip()
    print("Cleaned column names:", df_csv.columns.tolist())
    # --- END CLEANING ---

except FileNotFoundError:
    print("Error: file not found. Please provide your actual CSV file.")
    raise FileNotFoundError("CSV file not found, please check path.")

# --- Step 1: Forward-fill the parent columns ---
# This propagates the higher-level codes down to their children.
df_csv['Level 1'] = df_csv['Level 1'].replace('', pd.NA).ffill()
df_csv['Level 2'] = df_csv['Level 2'].replace('', pd.NA).ffill()
df_csv['Level 3'] = df_csv['Level 3'].replace('', pd.NA).ffill()
df_csv['Uniformat Level 4'] = df_csv['Uniformat Level 4'].replace('', pd.NA).ffill()
df_csv['Uniformat Level 5'] = df_csv['Uniformat Level 5'].replace('', pd.NA) # Level 5 should NOT ffill

print("\nDataFrame after forward-filling (first 15 rows):")
print(df_csv.head(15))

# --- Step 2: Determine the actual Uniformat code, its level, and its direct parent string for each row ---
processed_uniformat_data = []

for index, row in df_csv.iterrows():
    code = None
    level = None
    parent_code_str = None
    description = row['Description'].strip() # Ensure description is also stripped

    # Find the deepest non-empty code on this specific row that has a description.
    if pd.notna(row['Uniformat Level 5']):
        code = row['Uniformat Level 5'].strip()
        level = 5
        parent_code_str = row['Uniformat Level 4'].strip() if pd.notna(row['Uniformat Level 4']) else None
    elif pd.notna(row['Uniformat Level 4']):
        code = row['Uniformat Level 4'].strip()
        level = 4
        parent_code_str = row['Level 3'].strip() if pd.notna(row['Level 3']) else None
    elif pd.notna(row['Level 3']):
        code = row['Level 3'].strip()
        level = 3
        parent_code_str = row['Level 2'].strip() if pd.notna(row['Level 2']) else None
    elif pd.notna(row['Level 2']):
        code = row['Level 2'].strip()
        level = 2
        parent_code_str = row['Level 1'].strip() if pd.notna(row['Level 1']) else None
    elif pd.notna(row['Level 1']): # This catches Level 1 codes
        code = row['Level 1'].strip()
        level = 1
        parent_code_str = None # Top-level element has no parent

    if code and description: # Ensure we have both a code and a description
        processed_uniformat_data.append({
            'code': code,
            'description': description,
            'long_description': description, # Initial placeholder for LongDescription
            'level': level,
            'parent_code_str': parent_code_str, # Will resolve to ID later
            'standard_version': 'Custom' if level == 5 else 'ASTM E1557-09'
        })

# Convert to DataFrame for easier inspection/deduplication
df_processed_final = pd.DataFrame(processed_uniformat_data).drop_duplicates(subset=['code']).reset_index(drop=True)
print("\nFinal processed Uniformat data ready for DB insertion (deduplicated, first 15 rows):")
print(df_processed_final.head(15))

# --- Step 3: Insert into SQLite (Two-Pass for ParentCodeID resolution) ---
# Ensure 'conn' is your sqlite3 connection object and 'cursor' is active
cursor = conn.cursor()

# Get existing codes in DB to prevent re-inserting and to build ID map
cursor.execute("SELECT UniformatCode, UniformatCodeID FROM UniformatCode")
db_codes_map = {row[0]: row[1] for row in cursor.fetchall()}

# Sort records by level to ensure parents are inserted before children
records_to_insert = df_processed_final.to_dict(orient='records')
records_to_insert.sort(key=lambda x: x['level'])

# First pass: Insert all codes, collecting their DB IDs
print("\n--- First Pass: Inserting/Confirming Uniformat Codes and building ID map ---")
for record in records_to_insert:
    if record['code'] not in db_codes_map: # Only insert if not already in DB
        try:
            cursor.execute('''
                INSERT INTO UniformatCode (UniformatCode, Description, LongDescription, Level, StandardVersion)
                VALUES (?, ?, ?, ?, ?)
            ''', (record['code'], record['description'], record['long_description'], record['level'], record['standard_version']))
            conn.commit()
            db_codes_map[record['code']] = cursor.lastrowid # Store the newly generated ID
            # print(f"Inserted: {record['code']} (Level: {record['level']})")
        except sqlite3.IntegrityError as e:
            print(f"Skipping duplicate or error inserting {record['code']}: {e}")
            conn.rollback()
            # If unique constraint violation, it's already there, get its ID
            cursor.execute("SELECT UniformatCodeID FROM UniformatCode WHERE UniformatCode = ?", (record['code'],))
            result = cursor.fetchone()
            if result:
                db_codes_map[record['code']] = result[0]
            else:
                print(f"Failed to retrieve ID for existing code {record['code']}")
    # else:
        # print(f"Skipping (already in DB): {record['code']}")

# Second pass: Update ParentCodeID now that all codes and their IDs are mapped
print("\n--- Second Pass: Resolving ParentCodeIDs ---")
for record in records_to_insert:
    current_uniformat_id = db_codes_map.get(record['code'])
    parent_uniformat_id = db_codes_map.get(record['parent_code_str']) # Get ID of parent code string

    if current_uniformat_id: # Ensure the current code was successfully added/mapped
        if record['parent_code_str'] and parent_uniformat_id: # If it has a parent and we found its ID
            # Check if ParentCodeID is already correctly set to avoid unnecessary updates
            cursor.execute("SELECT ParentCodeID FROM UniformatCode WHERE UniformatCodeID = ?", (current_uniformat_id,))
            current_parent_id_in_db = cursor.fetchone()[0]

            if current_parent_id_in_db is None or current_parent_id_in_db != parent_uniformat_id:
                try:
                    cursor.execute('''
                        UPDATE UniformatCode
                        SET ParentCodeID = ?
                        WHERE UniformatCodeID = ?
                    ''', (parent_uniformat_id, current_uniformat_id))
                    conn.commit()
                    # print(f"Updated ParentCodeID for {record['code']} to {record['parent_code_str']}")
                except Exception as e:
                    print(f"Error updating ParentCodeID for {record['code']}: {e}")
        elif record['parent_code_str'] and parent_uniformat_id is None:
            # This means a parent code string was found but its ID couldn't be resolved (e.g., parent not in CSV or error)
            print(f"Warning: Parent code '{record['parent_code_str']}' for '{record['code']}' not found in DB_codes_map. ParentCodeID not set.")
        # Else: record['parent_code_str'] is None, which is correct for Level 1 elements

print("\nUniformat codes (Levels 1-5) from CSV inserted and parent IDs resolved in database.")

# --- Verification using pandas (avoids ipython-sql issues) ---
try:
    df_uniformat_hierarchy = pd.read_sql_query('''
        SELECT
            UC.UniformatCodeID,
            UC.UniformatCode,
            UC.Description,
            UC.LongDescription,
            UC.Level,
            UC.ParentCodeID,
            P.UniformatCode AS ParentCodeStr,
            P.Description AS ParentDesc
        FROM
            UniformatCode AS UC
        LEFT JOIN
            UniformatCode AS P ON UC.ParentCodeID = P.UniformatCodeID
        ORDER BY
            UC.UniformatCode;
    ''', conn)

    print("\nVerifying hierarchy in DB (using pandas - first 20 rows):")
    print(df_uniformat_hierarchy.head(20))
    print("\nIMPORTANT: Review 'ParentCodeID' and 'ParentCodeStr' columns for correct hierarchy. If still NULL for L2-L4, there's an issue in data processing.")

except Exception as e:
    print(f"Error fetching data to verify hierarchy: {e}")

CSV loaded successfully.
Cleaned column names: ['Level 1', 'Unnamed: 1', 'Level 2', 'Unnamed: 3', 'Level 3', 'Unnamed: 5', 'Uniformat Level 4', 'Uniformat Level 5', 'Description']

DataFrame after forward-filling (first 15 rows):
                 Level 1    Unnamed: 1         Level 2   Unnamed: 3  \
0   Major Group Elements                Group Elements                
1                      A  SUBSTRUCTURE  Group Elements                
2                      A                           A10  Foundations   
3                      A                           A10                
4                      A                           A10                
5                      A                           A10                
6                      A                           A10                
7                      A                           A10                
8                      A                           A10                
9                      A                           A10      

In [40]:
import pandas as pd
import sqlite3
import re
# ... other imports (fitz, google.generativeai, json, os) ...

# Ensure 'conn' and 'cursor' are defined from your database setup (Part 1.3)
# conn = sqlite3.connect('uniformat_data.db')
# cursor = conn.cursor()

# Define the expected column names after your manual cleanup
# These are the exact names you stated your CSV now has
EXPECTED_COLUMNS = [
    'Level 1 Major Group Elements',
    'Level 2 Group Elements',
    'Level 3 Individual Elements',
    'Uniformat Level 4',
    'Uniformat Level 5',
    'Description'
]

# --- Load and Initial Clean CSV ---
try:
    # Explicitly read with header=0 (default) assuming first row is header
    df_csv = pd.read_excel('Uniformat-II-Levels-1-to-5.xlsx', keep_default_na=False)
    print("CSV loaded successfully.")

    # Validate and clean column names. This is now simplified as they should be mostly correct.
    df_csv.columns = df_csv.columns.str.strip()
    
    # Optional: Check if the cleaned column names match expectations
    if not all(col in df_csv.columns for col in EXPECTED_COLUMNS):
        print(f"WARNING: CSV columns do not exactly match expected names. Found: {df_csv.columns.tolist()}")
        print(f"Expected: {EXPECTED_COLUMNS}")
        # Attempt to map to generic names for internal processing if needed, or raise error
        # For now, we proceed assuming they are close enough or the script will fail later.

    print("Cleaned column names:", df_csv.columns.tolist())
    print("\nFirst 5 rows of raw data (after header parsing):")
    print(df_csv.head()) # Verify that "A SUBSTRUCTURE" is the first data entry

except FileNotFoundError:
    print("Error: 'my_uniformat_structure.csv' not found. Please provide your actual CSV file.")
    raise # Re-raise the error to stop execution if file is missing

# --- Step 1: Forward-fill the parent columns ---
# We use the full, cleaned column names now.
df_csv['Level 1 Major Group Elements'] = df_csv['Level 1 Major Group Elements'].replace('', pd.NA).ffill()
df_csv['Level 2 Group Elements'] = df_csv['Level 2 Group Elements'].replace('', pd.NA).ffill()
df_csv['Level 3 Individual Elements'] = df_csv['Level 3 Individual Elements'].replace('', pd.NA).ffill()
df_csv['Uniformat Level 4'] = df_csv['Uniformat Level 4'].replace('', pd.NA).ffill()
df_csv['Uniformat Level 5'] = df_csv['Uniformat Level 5'].replace('', pd.NA) # Do NOT ffill Level 5

print("\nDataFrame after forward-filling (first 20 rows):")
print(df_csv.head(20))

# --- Step 2: Extract ALL unique Uniformat codes and their properties ---
extracted_uniformat_codes = []
processed_codes_set = set() # To track unique codes already added

# Helper function to add a code if it's new and determine its best description
def add_code_info(code_raw, level, parent_code_str_raw, description_source_value=None):
    code_str = str(code_raw).strip()
    if not code_str or code_str in processed_codes_set:
        return

    # Determine the best description for this code
    # Prioritize the explicitly provided description_source_value, then the code itself
    final_description = str(description_source_value).strip() if pd.notna(description_source_value) and str(description_source_value).strip() else code_str

    extracted_uniformat_codes.append({
        'code': code_str,
        'description': final_description,
        'long_description': final_description, # Initial placeholder, will be enriched later
        'level': level,
        'parent_code_str': str(parent_code_str_raw).strip() if pd.notna(parent_code_str_raw) else None,
        'standard_version': 'Custom' if level == 5 else 'ASTM E1557-09'
    })
    processed_codes_set.add(code_str)

# Iterate through each row of the forward-filled DataFrame
for index, row in df_csv.iterrows():
    # Helper variables for parent codes
    level1_code = row['Level 1 Major Group Elements'] if pd.notna(row['Level 1 Major Group Elements']) else None
    level2_code = row['Level 2 Group Elements'] if pd.notna(row['Level 2 Group Elements']) else None
    level3_code = row['Level 3 Individual Elements'] if pd.notna(row['Level 3 Individual Elements']) else None
    level4_code = row['Uniformat Level 4'] if pd.notna(row['Uniformat Level 4']) else None
    level5_code = row['Uniformat Level 5'] if pd.notna(row['Uniformat Level 5']) else None
    
    main_description_col = row['Description'] if pd.notna(row['Description']) else None


    # Process Level 1: 'A', 'B', etc.
    # Description usually comes from the 'Group Elements' column (e.g., "SUBSTRUCTURE")
    if level1_code:
        # Check if the Level 2 column contains a descriptive word (not just a code like "A10")
        description_for_l1 = None
        if level2_code and re.match(r'^[A-Z]{1}\d+$', str(level2_code).strip()) is None: # If L2 is NOT just a code like A10
            description_for_l1 = level2_code # Then L2 column contains the description for L1 (e.g. "SUBSTRUCTURE")
        elif main_description_col and str(main_description_col).strip(): # Fallback to main Description
            description_for_l1 = main_description_col
        
        add_code_info(
            level1_code,
            1,
            None, # No parent for Level 1
            description_for_l1 # Use derived description
        )

    # Process Level 2: 'A10', 'A20', etc.
    # Description often comes from the 'Individual Elements' column (e.g., "Foundations")
    if level2_code:
        description_for_l2 = None
        if level3_code and re.match(r'^[A-Z]{1}\d{3}$', str(level3_code).strip()) is None: # If L3 is NOT just a code like A1010
            description_for_l2 = level3_code # Then L3 column contains the description for L2 (e.g. "Foundations")
        elif main_description_col and str(main_description_col).strip():
            description_for_l2 = main_description_col
        
        add_code_info(
            level2_code,
            2,
            level1_code,
            description_for_l2 # Use derived description
        )

    # Process Level 3: 'A1010', 'A1020', etc.
    # Description often comes from the 'Uniformat Level 4' column (e.g., "STANDARD WALL FOUNDATIONS")
    if level3_code:
        description_for_l3 = None
        if level4_code and re.match(r'^[A-Z]{1}\d{5}$', str(level4_code).strip()) is None: # If L4 is NOT just a code like A101000
             description_for_l3 = level4_code # Then L4 column contains description (e.g. "STANDARD WALL FOUNDATIONS")
        elif main_description_col and str(main_description_col).strip():
            description_for_l3 = main_description_col

        add_code_info(
            level3_code,
            3,
            level2_code,
            description_for_l3 # Use derived description
        )

    # Process Uniformat Level 4: 'A101000', 'A101100', etc.
    # Description for L4 "General" items is typically in the 'Description' column,
    # or it might be in the L4 column itself if not "General".
    if level4_code:
        add_code_info(
            level4_code,
            4,
            level3_code,
            main_description_col if main_description_col else level4_code # Prioritize Description column, fallback to L4 code itself
        )

    # Process Uniformat Level 5: 'A101101', 'A101102', etc.
    # Description for L5 is always in the 'Description' column
    if level5_code:
        add_code_info(
            level5_code,
            5,
            level4_code,
            main_description_col # L5 always gets its description from the main 'Description' column
        )

# Convert to DataFrame and re-deduplicate
df_processed_final = pd.DataFrame(extracted_uniformat_codes).drop_duplicates(subset=['code']).reset_index(drop=True)

# --- DEBUG PRINT: Check the extracted codes after the new extraction logic ---
print("\n--- DEBUG: Extracted Uniformat Codes (first 40 rows after full extraction) ---")
print(df_processed_final.head(40).to_string())
print("Total unique codes extracted:", len(df_processed_final))
print("-------------------------------------------------------------------")


# --- Step 3: Insert into SQLite (Two-Pass for ParentCodeID resolution) ---
cursor = conn.cursor()

# Get existing codes in DB to prevent re-inserting and to build an ID map
cursor.execute("SELECT UniformatCode, UniformatCodeID FROM UniformatCode")
db_codes_map = {row[0]: row[1] for row in cursor.fetchall()}

# Sort records by level to ensure parents are processed/inserted before children
records_to_insert = df_processed_final.to_dict(orient='records')
records_to_insert.sort(key=lambda x: x['level'])

print("\n--- First Pass: Inserting/Confirming Uniformat Codes and building ID map ---")
for record in records_to_insert:
    code = record['code']
    if code not in db_codes_map:
        try:
            cursor.execute('''
                INSERT INTO UniformatCode (UniformatCode, Description, LongDescription, Level, StandardVersion)
                VALUES (?, ?, ?, ?, ?)
            ''', (code, record['description'], record['long_description'], record['level'], record['standard_version']))
            conn.commit()
            db_codes_map[code] = cursor.lastrowid
        except sqlite3.IntegrityError:
            conn.rollback()
            cursor.execute("SELECT UniformatCodeID FROM UniformatCode WHERE UniformatCode = ?", (code,))
            result = cursor.fetchone()
            if result:
                db_codes_map[code] = result[0]
            else:
                print(f"ERROR: Could not get ID for code {code} after failed insert.")

print("\n--- DEBUG: db_codes_map after First Pass (should contain all unique codes) ---")
print({k: v for k, v in list(db_codes_map.items())[:20]})
print("Total codes in map:", len(db_codes_map))
print("-------------------------------------------------------------------")


print("\n--- Second Pass: Resolving ParentCodeIDs ---")
for record in records_to_insert:
    code = record['code']
    current_uniformat_id = db_codes_map.get(code)
    parent_code_str = record['parent_code_str']
    parent_uniformat_id = None

    if parent_code_str:
        parent_uniformat_id = db_codes_map.get(parent_code_str)

    if current_uniformat_id:
        cursor.execute("SELECT ParentCodeID FROM UniformatCode WHERE UniformatCodeID = ?", (current_uniformat_id,))
        current_parent_id_in_db = cursor.fetchone()[0]

        if (parent_uniformat_id is not None and current_parent_id_in_db != parent_uniformat_id) or \
           (parent_uniformat_id is None and current_parent_id_in_db is not None):
            try:
                cursor.execute('''
                    UPDATE UniformatCode
                    SET ParentCodeID = ?
                    WHERE UniformatCodeID = ?
                ''', (parent_uniformat_id, current_uniformat_id))
                conn.commit()
            except Exception as e:
                print(f"Error updating ParentCodeID for {code} (Parent: {parent_code_str}): {e}")
        elif parent_code_str and parent_uniformat_id is None:
            print(f"Warning: Parent code '{parent_code_str}' for '{code}' not found in DB_codes_map. ParentCodeID not set. (Level: {record['level']})")

print("\nUniformat codes (Levels 1-5) from CSV inserted and parent IDs resolved in database.")

# --- Final Verification using pandas ---
try:
    df_uniformat_hierarchy = pd.read_sql_query('''
        SELECT
            UC.UniformatCodeID,
            UC.UniformatCode,
            UC.Description,
            UC.LongDescription,
            UC.Level,
            UC.ParentCodeID,
            P.UniformatCode AS ParentCodeStr,
            P.Description AS ParentDesc
        FROM
            UniformatCode AS UC
        LEFT JOIN
            UniformatCode AS P ON UC.ParentCodeID = P.UniformatCodeID
        ORDER BY
            UC.UniformatCode;
    ''', conn)

    print("\n--- Final Verification: Hierarchy in DB (using pandas - first 50 rows) ---")
    print(df_uniformat_hierarchy.head(50).to_string())
    print("\nIMPORTANT: Carefully review 'ParentCodeID' and 'ParentCodeStr' columns. They should now be populated correctly for all levels (except Level 1).")
    print("-------------------------------------------------------------------")

except Exception as e:
    print(f"Error fetching data to verify hierarchy: {e}")

CSV loaded successfully.
Cleaned column names: ['Level 1 Major Group Elements', 'Unnamed: 1', 'Level 2 Group Elements', 'Unnamed: 3', 'Level 3 Individual Elements', 'Unnamed: 5', 'Uniformat Level 4', 'Uniformat Level 5', 'Description']

First 5 rows of raw data (after header parsing):
  Level 1 Major Group Elements    Unnamed: 1 Level 2 Group Elements  \
0                                                                     
1                            A  SUBSTRUCTURE                          
2                                                               A10   
3                                                                     
4                                                                     

    Unnamed: 3 Level 3 Individual Elements                 Unnamed: 5  \
0                                                                       
1                                                                       
2  Foundations                                                      

In [42]:
import pandas as pd
import sqlite3
import re
# ... other imports (fitz, google.generativeai, json, os) ...

# Define the Excel filename
EXCEL_FILENAME = 'Uniformat-II-Levels-1-to-5.xlsx'

# --- Define the mapping from raw Excel column names to internal, consistent names ---
# This is crucial. It reflects your latest clarification of the header structure.
COLUMN_MAP = {
    'Level 1': 'Level_1_Code',
    'Major Group Elements': 'Level_1_Description', # This column contains the description for Level 1 codes
    'Level 2': 'Level_2_Code',
    'Group Elements': 'Level_2_Description',   # This column contains the description for Level 2 codes
    'Level 3': 'Level_3_Code',
    'Individual Elements': 'Level_3_Description', # This column contains the description for Level 3 codes
    'Uniformat Level 4': 'Level_4_Code',
    'Uniformat Level 5': 'Level_5_Code',
    'Description': 'Level_5_and_L4_General_Description' # This column typically holds L5 descriptions or L4 "General"
}

# --- Load and Initial Data Preparation ---
try:
    # Read the Excel file, assuming a single header row (0-indexed)
    df_excel = pd.read_excel(EXCEL_FILENAME, header=0, keep_default_na=False)
    print(f"Excel file '{EXCEL_FILENAME}' loaded successfully with single-level headers.")

    # Strip any whitespace from the raw column names
    df_excel.columns = df_excel.columns.str.strip()

    # Rename columns according to our defined map
    # Create a new DataFrame with just the columns we care about, renamed.
    df_processed = pd.DataFrame()
    for raw_col, new_col in COLUMN_MAP.items():
        if raw_col in df_excel.columns:
            df_processed[new_col] = df_excel[raw_col]
        else:
            print(f"ERROR: Expected column '{raw_col}' not found in Excel data. Please check Excel headers.")
            raise ValueError(f"Missing expected column: {raw_col}")

    # Forward-fill the code columns. Descriptions are handled separately.
    df_processed['Level_1_Code'] = df_processed['Level_1_Code'].replace('', pd.NA).ffill()
    df_processed['Level_2_Code'] = df_processed['Level_2_Code'].replace('', pd.NA).ffill()
    df_processed['Level_3_Code'] = df_processed['Level_3_Code'].replace('', pd.NA).ffill()
    df_processed['Level_4_Code'] = df_processed['Level_4_Code'].replace('', pd.NA).ffill()
    df_processed['Level_5_Code'] = df_processed['Level_5_Code'].replace('', pd.NA) # Level 5 is the deepest, do NOT ffill

    print("\nDataFrame after column renaming and forward-filling (first 20 rows):")
    print(df_processed.head(20))

except FileNotFoundError:
    print(f"Error: '{EXCEL_FILENAME}' not found. Please ensure the file is in the correct directory.")
    raise
except Exception as e:
    print(f"An error occurred during Excel loading or initial processing: {e}")
    raise


# --- Step 2: Extract ALL unique Uniformat codes and their properties ---
extracted_uniformat_codes = []
processed_codes_set = set() # To track unique codes already added, preventing duplicates

# Helper function to add a code if it's new and determine its best description
def add_code_info(code_raw, level, parent_code_str_raw, description_value=None):
    code_str = str(code_raw).strip()
    if not code_str or code_str in processed_codes_set:
        return

    # Use the provided description_value. If empty, fall back to the code itself.
    final_description = str(description_value).strip() if pd.notna(description_value) and str(description_value).strip() else code_str

    extracted_uniformat_codes.append({
        'code': code_str,
        'description': final_description,
        'long_description': final_description, # Initial placeholder, will be enriched later
        'level': level,
        'parent_code_str': str(parent_code_str_raw).strip() if pd.notna(parent_code_str_raw) else None,
        'standard_version': 'Custom' if level == 5 else 'ASTM E1557-09'
    })
    processed_codes_set.add(code_str)

# Iterate through each row of the processed DataFrame
for index, row in df_processed.iterrows():
    # Extract codes and descriptions using the new internal column names
    level1_code = row['Level_1_Code']
    level1_desc = row['Level_1_Description']

    level2_code = row['Level_2_Code']
    level2_desc = row['Level_2_Description']

    level3_code = row['Level_3_Code']
    level3_desc = row['Level_3_Description']

    level4_code = row['Level_4_Code']
    level5_code = row['Level_5_Code'] # This is not forward-filled past its own row
    
    # This description column is specifically for L5 or L4 'General' items
    level5_and_l4_general_desc = row['Level_5_and_L4_General_Description']

    # Process Level 1
    if pd.notna(level1_code):
        # L1 description comes from 'Major Group Elements' column
        add_code_info(level1_code, 1, None, level1_desc)

    # Process Level 2
    if pd.notna(level2_code):
        # L2 description comes from 'Group Elements' column
        add_code_info(level2_code, 2, level1_code, level2_desc)

    # Process Level 3
    if pd.notna(level3_code):
        # L3 description comes from 'Individual Elements' column
        add_code_info(level3_code, 3, level2_code, level3_desc)

    # Process Uniformat Level 4
    if pd.notna(level4_code):
        # L4 description comes from 'Level_5_and_L4_General_Description' column
        add_code_info(level4_code, 4, level3_code, level5_and_l4_general_desc)

    # Process Uniformat Level 5
    if pd.notna(level5_code):
        # L5 description comes from 'Level_5_and_L4_General_Description' column
        add_code_info(level5_code, 5, level4_code, level5_and_l4_general_desc)


# Convert the list of dictionaries to a DataFrame
df_processed_final = pd.DataFrame(extracted_uniformat_codes).drop_duplicates(subset=['code']).reset_index(drop=True)

# --- DEBUG PRINT: Check the extracted codes after the new extraction logic ---
print("\n--- DEBUG: Extracted Uniformat Codes (first 40 rows after full extraction) ---")
print(df_processed_final.head(40).to_string())
print("Total unique codes extracted:", len(df_processed_final))
print("-------------------------------------------------------------------")


# --- Step 3: Insert into SQLite (Two-Pass for ParentCodeID resolution) ---
cursor = conn.cursor()

# Get existing codes in DB to prevent re-inserting and to build an ID map
cursor.execute("SELECT UniformatCode, UniformatCodeID FROM UniformatCode")
db_codes_map = {row[0]: row[1] for row in cursor.fetchall()}

# Sort records by level to ensure parents are processed/inserted before children
records_to_insert = df_processed_final.to_dict(orient='records')
records_to_insert.sort(key=lambda x: x['level'])

print("\n--- First Pass: Inserting/Confirming Uniformat Codes and building ID map ---")
for record in records_to_insert:
    code = record['code']
    if code not in db_codes_map:
        try:
            cursor.execute('''
                INSERT INTO UniformatCode (UniformatCode, Description, LongDescription, Level, StandardVersion)
                VALUES (?, ?, ?, ?, ?)
            ''', (code, record['description'], record['long_description'], record['level'], record['standard_version']))
            conn.commit()
            db_codes_map[code] = cursor.lastrowid
        except sqlite3.IntegrityError:
            conn.rollback()
            cursor.execute("SELECT UniformatCodeID FROM UniformatCode WHERE UniformatCode = ?", (code,))
            result = cursor.fetchone()
            if result:
                db_codes_map[code] = result[0]
            else:
                print(f"ERROR: Could not get ID for code {code} after failed insert.")

print("\n--- DEBUG: db_codes_map after First Pass (should contain all unique codes) ---")
print({k: v for k, v in list(db_codes_map.items())[:20]})
print("Total codes in map:", len(db_codes_map))
print("-------------------------------------------------------------------")


print("\n--- Second Pass: Resolving ParentCodeIDs ---")
for record in records_to_insert:
    code = record['code']
    current_uniformat_id = db_codes_map.get(code)
    parent_code_str = record['parent_code_str']
    parent_uniformat_id = None

    if parent_code_str:
        parent_uniformat_id = db_codes_map.get(parent_code_str)

    if current_uniformat_id:
        cursor.execute("SELECT ParentCodeID FROM UniformatCode WHERE UniformatCodeID = ?", (current_uniformat_id,))
        current_parent_id_in_db = cursor.fetchone()[0]

        if (parent_uniformat_id is not None and current_parent_id_in_db != parent_uniformat_id) or \
           (parent_uniformat_id is None and current_parent_id_in_db is not None):
            try:
                cursor.execute('''
                    UPDATE UniformatCode
                    SET ParentCodeID = ?
                    WHERE UniformatCodeID = ?
                ''', (parent_uniformat_id, current_uniformat_id))
                conn.commit()
            except Exception as e:
                print(f"Error updating ParentCodeID for {code} (Parent: {parent_code_str}): {e}")
        elif parent_code_str and parent_uniformat_id is None:
            print(f"Warning: Parent code '{parent_code_str}' for '{code}' not found in DB_codes_map. ParentCodeID not set. (Level: {record['level']})")

print("\nUniformat codes (Levels 1-5) from Excel inserted and parent IDs resolved in database.")

# --- Final Verification using pandas ---
try:
    df_uniformat_hierarchy = pd.read_sql_query('''
        SELECT
            UC.UniformatCodeID,
            UC.UniformatCode,
            UC.Description,
            UC.LongDescription,
            UC.Level,
            UC.ParentCodeID,
            P.UniformatCode AS ParentCodeStr,
            P.Description AS ParentDesc
        FROM
            UniformatCode AS UC
        LEFT JOIN
            UniformatCode AS P ON UC.ParentCodeID = P.UniformatCodeID
        ORDER BY
            UC.UniformatCode;
    ''', conn)

    print("\n--- Final Verification: Hierarchy in DB (using pandas - first 50 rows) ---")
    print(df_uniformat_hierarchy.head(50).to_string())
    print("\nIMPORTANT: Carefully review 'ParentCodeID' and 'ParentCodeStr' columns. They should now be populated correctly for all levels (except Level 1).")
    print("-------------------------------------------------------------------")

except Exception as e:
    print(f"Error fetching data to verify hierarchy: {e}")

Excel file 'Uniformat-II-Levels-1-to-5.xlsx' loaded successfully with single-level headers.

DataFrame after column renaming and forward-filling (first 20 rows):
   Level_1_Code Level_1_Description Level_2_Code    Level_2_Description  \
0             A        SUBSTRUCTURE         <NA>                          
1             A                              A10            Foundations   
2             A                              A10                          
3             A                              A10                          
4             A                              A10                          
5             A                              A10                          
6             A                              A10                          
7             A                              A10                          
8             A                              A10                          
9             A                              A10                          
10           